In [1]:
%matplotlib qt

from numpy import *
from matplotlib.pyplot import *
import pickle
from matplotlib2tikz import save as tikz_save
from quadcopter.util import *

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 160


def to_deg(rad):
    return rad / np.pi * 180.0


def load_results(file_name):
    complete_path = f"C:\\dev\\mhe_pymoskito\\ballbeam\\results\\simulation\\{file_name}.pmr"
    with open(complete_path, "rb") as f:
        return pickle.load(f)['results']
    

class ExperimentData:
    def __init__(self, ex_id, raw):
        self.ex_id = ex_id
        self.raw = raw
    
    @property
    def time(self):
        return self.raw['time']
   
    @property 
    def solver(self):
        return self.raw['Solver'].T
    
    @property
    def observer(self):
        return self.raw['Observer'][:, :4].T
    
    @property
    def covariance(self):
        return self.raw['Observer'][:, 4:].T
    
    @property
    def measurement(self):
        return self.raw['Model'].T
    
    
def load_experiment(ex_id, resample_steps=None, start_time=None, end_time=None) -> ExperimentData:
    raw = load_results(ex_id)
    
    start_index = None
    end_index = None
    
    if start_time is not None:
        start_index = time_to_index(start_time, raw['time'])
        
    if end_time is not None:
        end_index = time_to_index(end_time, raw['time'])
    
    raw['time'] = raw['time'][start_index:end_index:resample_steps]
    raw['Solver'] = raw['Solver'][start_index:end_index:resample_steps]
    raw['Observer'] = raw['Observer'][start_index:end_index:resample_steps]
    raw['Model'] = raw['Model'][start_index:end_index:resample_steps]
    return ExperimentData(ex_id, raw)


def plot_with_sd(t, measurement, real, sd, sigma_size=3):
    lower_limit = real - sigma_size*sd
    upper_limit = real + sigma_size*sd
    
    plot(t, measurement, color='blue')
    plot(t, real, '--', color='green')
    plot(t, lower_limit, color='red')
    plot(t, upper_limit, color='red')


time = load_results('ekf_with_noise')['time']
ekf = load_results('ekf_with_noise')['Observer']
ekf_real = load_results('ekf_with_noise')['Solver']
ekf_meas = load_results('ekf_with_noise')['Model']
slsqp = load_results('mhe_with_noise')['Observer']
slsqp_real = load_results('mhe_with_noise')['Solver']
slsqp_meas = load_results('mhe_with_noise')['Model']

In [3]:
ex1 = load_experiment('511')
ex2 = load_experiment('512')
combined_plot = True

figure()
plot(ex1.time, ex1.measurement[0])

if combined_plot:
    figure("Combined plot", figsize=(18, 10))

for i in range(4):
    if combined_plot:
        subplot(421 + 2*i)
    else:
        figure(f"State {i+1} observers")
        
    plot(ex1.time, ex1.solver[i], '--', c="blue")
    plot(ex1.time, ex1.observer[i], c="red")
    plot(ex1.time, ex2.observer[i], c="green")
    legend(["reference", ex1.ex_id, ex2.ex_id])
    title(f"State {i+1} observers")
    grid()
    
    if combined_plot:
        subplot(421 + 2*i + 1)
    else:
        figure(f"State {i+1} errors")
        
    plot(ex1.time, ex1.observer[i] - ex1.solver[i], c="red")
    plot(ex1.time, ex2.observer[i] - ex2.solver[i], c="green")
    mse1 = sum([e**2 for e in ex1.observer[i] - ex1.solver[i]]) / ex1.time.size
    mse2 = sum([e**2 for e in ex2.observer[i] - ex2.solver[i]]) / ex2.time.size
    legend([f"{ex1.ex_id} - mse={mse1}", f"{ex2.ex_id} - mse={mse2}"])
    title(f"State {i+1} errors")
    grid()

show()

C:\dev\Python\studienarbeit\lib\site-packages\matplotlib\cbook\deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [64]:
ekf_exs = [load_experiment('511', 20), load_experiment('531', 20), load_experiment('541', 20)]
mhe_exs = [load_experiment('512', 20), load_experiment('532', 20), load_experiment('542', 20)]
ref = ekf_exs[0]
rows = len(ekf_exs)

figure()
for i in range(rows):
    subplot(rows, 2, 2*i + 1)
    plot(ref.time, to_deg(ref.solver[2]), color='black')
    plot(ref.time, to_deg(ekf_exs[i].observer[2]), color='r', linestyle='--')
    plot(ref.time, to_deg(mhe_exs[i].observer[2]), color='g', linestyle='--')
    xlabel('$t$ (s)')
    ylabel('$\\theta$ (deg)')
    grid()
    
    subplot(rows, 2, 2*i + 2)
    plot(ref.time, to_deg(ekf_exs[i].observer[2] - ref.solver[2]), color='r', linestyle='--')
    plot(ref.time, to_deg(mhe_exs[i].observer[2] - ref.solver[2]), color='g', linestyle='--')
    xlabel('$t$ (s)')
    ylabel('$\\Delta_\\theta$ (deg)')
    grid()

tikz_save('plots/step_width.tex', figureheight='\\figureheight', figurewidth='\\figurewidth')
show()

Please add the following lines to your LaTeX preamble:

\usepackage[utf8]{inputenc}
\usepackage{fontspec} % This line only for XeLaTeX and LuaLaTeX
\usepackage{pgfplots}
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label tex

Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter


In [73]:
figure()
plot([1, 4, 8], [7.73, 52.48, 145.78], color='b', marker='o')
plot([1, 4, 8, 40], [2.37, 3.16, 3.56, 12.42], color='g', marker='o')
axhline(5, color='black', linestyle='--')
axhline(0.2, color='r', linestyle='--')
xlabel('Horizontweite')
ylabel('Rechenzeit für Beobachter (ms)')

tikz_save('plots/performance.tex', figureheight='\\figureheight', figurewidth='\\figurewidth')
show()

Please add the following lines to your LaTeX preamble:

\usepackage[utf8]{inputenc}
\usepackage{fontspec} % This line only for XeLaTeX and LuaLaTeX
\usepackage{pgfplots}
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter


In [23]:
ekf = load_experiment('421', end_time=8.0)
mhe1 = load_experiment('422', end_time=8.0)
mhe8 = load_experiment('424', end_time=8.0)
ref = ekf

figure()

subplot(4, 2, 1)
plot(ref.time, ref.solver[0], color='black')
plot(ref.time, ekf.observer[0], color='red', linestyle='--')
plot(ref.time, mhe1.observer[0], color='yellowgreen', linestyle='--')
plot(ref.time, mhe8.observer[0], color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$r$ (m)')

subplot(4, 2, 2)
plot(ref.time, ekf.observer[0] - ref.solver[0], color='red', linestyle='--')
plot(ref.time, mhe1.observer[0] - ref.solver[0], color='yellowgreen', linestyle='--')
plot(ref.time, mhe8.observer[0] - ref.solver[0], color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\Delta_r$ (m)')

subplot(4, 2, 3)
plot(ref.time, ref.solver[1], color='black')
plot(ref.time, ekf.observer[1], color='red', linestyle='--')
plot(ref.time, mhe1.observer[1], color='yellowgreen', linestyle='--')
plot(ref.time, mhe8.observer[1], color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$v$ (m/s)')

subplot(4, 2, 4)
plot(ref.time, ekf.observer[1] - ref.solver[1], color='red', linestyle='--')
plot(ref.time, mhe1.observer[1] - ref.solver[1], color='yellowgreen', linestyle='--')
plot(ref.time, mhe8.observer[1] - ref.solver[1], color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\Delta_v$ (m/s)')

subplot(4, 2, 5)
plot(ref.time, to_deg(ref.solver[2]), color='black')
plot(ref.time, to_deg(ekf.observer[2]), color='red', linestyle='--')
plot(ref.time, to_deg(mhe1.observer[2]), color='yellowgreen', linestyle='--')
plot(ref.time, to_deg(mhe8.observer[2]), color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\theta$ (deg)')

subplot(4, 2, 6)
plot(ref.time, to_deg(ekf.observer[2] - ref.solver[2]), color='red', linestyle='--')
plot(ref.time, to_deg(mhe1.observer[2] - ref.solver[2]), color='yellowgreen', linestyle='--')
plot(ref.time, to_deg(mhe8.observer[2] - ref.solver[2]), color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\Delta_\\theta$ (deg)')

subplot(4, 2, 7)
plot(ref.time, to_deg(ref.solver[3]), color='black')
plot(ref.time, to_deg(ekf.observer[3]), color='red', linestyle='--')
plot(ref.time, to_deg(mhe1.observer[3]), color='yellowgreen', linestyle='--')
plot(ref.time, to_deg(mhe8.observer[3]), color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\omega$ (deg/s)')

subplot(4, 2, 8)
plot(ref.time, to_deg(ekf.observer[3] - ref.solver[3]), color='red', linestyle='--')
plot(ref.time, to_deg(mhe1.observer[3] - ref.solver[3]), color='yellowgreen', linestyle='--')
plot(ref.time, to_deg(mhe8.observer[3] - ref.solver[3]), color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\Delta_{\\omega}$ (deg/s)')

tikz_save('plots/medium_initial_error.tex', figureheight='\\figureheight', figurewidth='\\figurewidth')
show()

Please add the following lines to your LaTeX preamble:

\usepackage[utf8]{inputenc}
\usepackage{fontspec} % This line only for XeLaTeX and LuaLaTeX
\usepackage{pgfplots}
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label tex


Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter


In [22]:
ekf = load_experiment('431')
mhe1 = load_experiment('432')
mhe8 = load_experiment('434')
ref = ekf

figure()

subplot(4, 2, 1)
plot(ref.time, ref.solver[0], color='black')
plot(ref.time, ekf.observer[0], color='red', linestyle='--')
plot(ref.time, mhe1.observer[0], color='yellowgreen', linestyle='--')
plot(ref.time, mhe8.observer[0], color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$r$ (m)')

subplot(4, 2, 2)
plot(ref.time, ekf.observer[0] - ref.solver[0], color='red', linestyle='--')
plot(ref.time, mhe1.observer[0] - ref.solver[0], color='yellowgreen', linestyle='--')
plot(ref.time, mhe8.observer[0] - ref.solver[0], color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\Delta_r$ (m)')

subplot(4, 2, 3)
plot(ref.time, ref.solver[1], color='black')
plot(ref.time, ekf.observer[1], color='red', linestyle='--')
plot(ref.time, mhe1.observer[1], color='yellowgreen', linestyle='--')
plot(ref.time, mhe8.observer[1], color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$v$ (m/s)')

subplot(4, 2, 4)
plot(ref.time, ekf.observer[1] - ref.solver[1], color='red', linestyle='--')
plot(ref.time, mhe1.observer[1] - ref.solver[1], color='yellowgreen', linestyle='--')
plot(ref.time, mhe8.observer[1] - ref.solver[1], color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\Delta_v$ (m/s)')

subplot(4, 2, 5)
plot(ref.time, to_deg(ref.solver[2]), color='black')
plot(ref.time, to_deg(ekf.observer[2]), color='red', linestyle='--')
plot(ref.time, to_deg(mhe1.observer[2]), color='yellowgreen', linestyle='--')
plot(ref.time, to_deg(mhe8.observer[2]), color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\theta$ (deg)')

subplot(4, 2, 6)
plot(ref.time, to_deg(ekf.observer[2] - ref.solver[2]), color='red', linestyle='--')
plot(ref.time, to_deg(mhe1.observer[2] - ref.solver[2]), color='yellowgreen', linestyle='--')
plot(ref.time, to_deg(mhe8.observer[2] - ref.solver[2]), color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\Delta_\\theta$ (deg)')

subplot(4, 2, 7)
plot(ref.time, to_deg(ref.solver[3]), color='black')
plot(ref.time, to_deg(ekf.observer[3]), color='red', linestyle='--')
plot(ref.time, to_deg(mhe1.observer[3]), color='yellowgreen', linestyle='--')
plot(ref.time, to_deg(mhe8.observer[3]), color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\omega$ (deg/s)')

subplot(4, 2, 8)
plot(ref.time, to_deg(ekf.observer[3] - ref.solver[3]), color='red', linestyle='--')
plot(ref.time, to_deg(mhe1.observer[3] - ref.solver[3]), color='yellowgreen', linestyle='--')
plot(ref.time, to_deg(mhe8.observer[3] - ref.solver[3]), color='seagreen', linestyle='--')
grid()
xlabel('$t$ (s)')
ylabel('$\\Delta_{\\omega}$ (deg/s)')

tikz_save('plots/large_initial_error.tex', figureheight='\\figureheight', figurewidth='\\figurewidth')
show()

Please add the following lines to your LaTeX preamble:

\usepackage[utf8]{inputenc}
\usepackage{fontspec} % This line only for XeLaTeX and LuaLaTeX
\usepackage{pgfplots}
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label tex

Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
